In [ ]:
# -*- coding: utf-8 -*-
"""
INEGI API – Llegadas/Salidas (Turismo) – ADAPTADO A PLOTLY/STREAMLIT
"""

import requests, pandas as pd, json, time, warnings, os
import plotly.graph_objects as go
import streamlit as st 

# ---------------------------------------------
# 1. Adaptación de Estilo para Streamlit
# ---------------------------------------------
DEFAULT_FONT = "Aptos Light"
# Orden de colores por defecto: [Llegadas (Verde), Salidas (Naranja)]
DEFAULT_PALETTE = ["#889064", "#ff9f18"]

# Obtener variables inyectadas de Streamlit o usar valores por defecto
PALETTE = globals().get('active_palette', DEFAULT_PALETTE)
FONT = globals().get('active_font', DEFAULT_FONT)

# Asignar variables de estilo
COLOR_LLEGADAS = PALETTE[0] if len(PALETTE) > 0 else DEFAULT_PALETTE[0]
COLOR_SALIDAS = PALETTE[1] if len(PALETTE) > 1 else DEFAULT_PALETTE[1]
FONT_FAMILY = FONT


def plot_bars_plotly(ann_df, title, last_years=15):
    """Genera la figura de Plotly basada en el dataframe anual."""
    if ann_df.empty:
        return None

    d = ann_df.sort_values("year")
    if isinstance(last_years, int) and len(d) > last_years:
        d = d.tail(last_years)

    fig = go.Figure()
    color = COLOR_LLEGADAS if "Llegadas" in title else COLOR_SALIDAS

    fig.add_trace(go.Bar(
        x=d["year"].astype(str),
        y=d["value"],
        marker_color=color,
        text=[f"{v:,.0f}" for v in d["value"]],
        textposition="outside"
    ))

    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center'
        ),
        xaxis_title="Año",
        yaxis_title="Número de turistas (suma anual)",
        font=dict(family=FONT_FAMILY, size=14),
        template="plotly_white",
        margin=dict(l=40, r=40, t=80, b=100)
    )
    fig.update_yaxes(tickformat=",.0f")

    fig.add_annotation(
        text="Fuente: INEGI (Banco de Información Económica)",
        xref="paper", yref="paper",
        x=0,      
        y=-0.15,   
        showarrow=False,
        xanchor='left',
        yanchor='top',
        font=dict(size=12, color="gray", family=FONT_FAMILY)
    )

    return fig

def prepare_table_data(df, last_years=15):
    """Prepara el DataFrame para mostrarse en la tabla (mismo filtro que la gráfica)."""
    if df.empty: return pd.DataFrame()
    
    # 1. Ordenar y filtrar igual que la gráfica
    d = df.sort_values("year")
    if isinstance(last_years, int) and len(d) > last_years:
        d = d.tail(last_years)
    
    # 2. Seleccionar columnas y renombrar para presentación
    out = d[["year", "value"]].copy()
    out.columns = ["Año", "Número de Turistas"]
    
    # 3. Invertir orden para que el año más reciente salga arriba en la tabla (opcional, pero usualmente preferido en tablas)
    # Si prefieres orden cronológico (igual que la gráfica), comenta la línea siguiente:
    out = out.sort_values("Año", ascending=False)
    
    return out


# ── Parámetros de la API ──────────────────────────────────────────────────────
TOKEN = "460ccba7-40b7-08a2-47dd-7301e6b6fbbc"
IDS = ["6207123161","6207123163","6207123168","6207123170","6207123178"]
BASE = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml"
LANG = "es"
AREAS = ["00","0700"] 
BANKS = ["BISE","BIE"]
VERS = "2.0"
TIMEOUT = 30
PLOT_LAST_YEARS = 15

# ── Funciones de Consulta API ─────────────────────────────────────────────────

def http_get(url):
    r = requests.get(url, timeout=TIMEOUT)
    r.raise_for_status()
    return r

def fetch_indicator_last(ind_id: str):
    last_ex = None
    for bank in BANKS:
        for area in AREAS:
            url = f"{BASE}/INDICATOR/{ind_id}/{LANG}/{area}/true/{bank}/{VERS}/{TOKEN}?type=json"
            try:
                j = http_get(url).json()
                ser = j.get("Series", [])
                if not ser: continue
                s = ser[0]; obs = s.get("OBSERVATIONS", [])
                if not obs: continue
                last = obs[-1]
                return {
                    "id": ind_id, "bank": bank, "area": area, "raw": j,
                    "period": last.get("TIME_PERIOD",""), "value": last.get("OBS_VALUE",""),
                    "unit_id": s.get("UNIT",""), "name": s.get("INDICATOR_NAME","")
                }
            except Exception as e:
                last_ex = e
                time.sleep(0.4)
    raise RuntimeError(f"No pude leer último dato de {ind_id}. Último error: {last_ex}")

def fetch_metadata(ind_id: str):
    url = f"{BASE}/METADATA/INDICATOR/{ind_id}/{LANG}/{VERS}/{TOKEN}?type=json"
    try:
        j = http_get(url).json()
        s = (j.get("Series") or [{}])[0]
        return {
            "id": ind_id,
            "name_meta": s.get("INDICATOR_NAME") or s.get("TITLE") or "",
            "unit_meta": s.get("UNIT_NAME") or s.get("UNIT") or "",
            "raw_meta": j
        }
    except Exception as e:
        return {"id": ind_id, "name_meta": "", "unit_meta": "", "raw_meta": {"error": str(e)}}

def classify(name_all: str, unit_name: str, unit_id: str, value):
    nm = (name_all or "").lower()
    un = (unit_name or "").lower()
    if "dólar" in un or "usd" in un or "mdd" in un or (isinstance(value,(int,float)) and value and value>1e7 and unit_id in ("21","022","$")):
        return "Monto $ (divisas/gasto) – no conteo"
    if "salid" in nm or ("residentes" in nm and ("exterior" in nm or "extranj" in nm)):
        return "EGRESOS – Salidas (residentes al exterior)"
    if "turistas internacionales" in nm or ("turistas" in nm and "internación" in nm):
        return "INGRESOS – Llegadas (turistas internacionales)"
    if "visitantes internacionales" in nm:
        return "INGRESOS – Visitantes (turistas + excursionistas)"
    if unit_id in ("1011","1000","1001"):
        return "Conteo (personas) – por revisar nombre"
    return "Por revisar"

def fetch_series_full(ind_id: str):
    for bank in BANKS:
        for area in AREAS:
            url = f"{BASE}/INDICATOR/{ind_id}/{LANG}/{area}/false/{bank}/{VERS}/{TOKEN}?type=json"
            try:
                j = http_get(url).json()
                ser = j.get("Series", [])
                if not ser: continue
                s = ser[0]; obs = s.get("OBSERVATIONS", [])
                if not obs: continue
                recs=[]
                for o in obs:
                    per = o.get("TIME_PERIOD","")
                    val = o.get("OBS_VALUE",None)
                    if per and val is not None:
                        if "/" in per: dt = pd.to_datetime(per.replace("/","-")+"-01")
                        else: dt = pd.to_datetime(per+"-01-01")
                        recs.append({"date": dt, "value": float(val)})
                df = pd.DataFrame(recs).sort_values("date").reset_index(drop=True)
                df["year"] = df["date"].dt.year
                if df.groupby("year")["date"].count().median() >= 4:
                    ann = df.groupby("year", as_index=False)["value"].sum()
                else:
                    ann = df.groupby("year", as_index=False)["value"].sum()
                ann["id"] = ind_id
                return df, ann
            except Exception:
                time.sleep(0.4)
                continue
    return pd.DataFrame(columns=["date","value"]), pd.DataFrame(columns=["year","value"])


# ------------------------------------------------------------------------------
# ── FUNCIÓN PRINCIPAL DE EJECUCIÓN (Usamos st.cache_data) ─────────────────────
# ------------------------------------------------------------------------------
@st.cache_data(ttl=3600)
def fetch_and_plot_turismo():

    # 1) Validación últimos datos + metadatos
    rows=[]
    for ind in IDS:
        try:
            last = fetch_indicator_last(ind)
            meta = fetch_metadata(ind)
            full_name = meta["name_meta"] or last["name"] or "(sin nombre)"
            unit_read = meta["unit_meta"] or last["unit_id"] or ""
            clasif = classify(
                full_name, unit_read, str(last.get("unit_id","")),
                float(last["value"]) if str(last["value"]).replace('.','',1).isdigit() else None
            )
            rows.append({
                "id": ind, "indicador": full_name, "último_valor": last["value"], "clasificacion": clasif
            })
        except RuntimeError:
            pass

    df_last = pd.DataFrame(rows)

    # 2) Selección automática de IDs
    cont = df_last[df_last["clasificacion"].str.contains("Conteo|INGRESOS|EGRESOS", na=False)].copy()
    if cont.empty: raise ValueError("No se detectaron candidatos de conteo.")

    id_llegadas = cont[cont["clasificacion"].str.contains("INGRESOS", na=False)]["id"].iloc[0] if not cont[cont["clasificacion"].str.contains("INGRESOS", na=False)].empty else None
    id_salidas = cont[cont["clasificacion"].str.contains("EGRESOS", na=False)]["id"].iloc[0] if not cont[cont["clasificacion"].str.contains("EGRESOS", na=False)].empty else None

    if not id_llegadas: id_llegadas = cont.sort_values("último_valor", ascending=False)["id"].iloc[0]
    if not id_salidas:
        rest = cont[cont["id"]!=id_llegadas]
        if not rest.empty: id_salidas = rest.sort_values("último_valor", ascending=False)["id"].iloc[0]

    # 3) Descargar series completas
    _, a_lleg = fetch_series_full(id_llegadas)
    _, a_sali = fetch_series_full(id_salidas)

    # 4) Generar figuras
    fig_llegadas = plot_bars_plotly(a_lleg, f"México – Llegadas de turistas internacionales", PLOT_LAST_YEARS)
    fig_salidas = plot_bars_plotly(a_sali, f"México – Salidas de residentes al extranjero", PLOT_LAST_YEARS)

    return fig_llegadas, fig_salidas, a_lleg, a_sali

# ------------------------------------------------------------------------------
# ── EJECUCIÓN Y VISUALIZACIÓN EN STREAMLIT ─────────────────────────────────────
# ------------------------------------------------------------------------------

st.title("Turismo: Llegadas y Salidas Internacionales (INEGI)")

if st.button("Recargar datos (INEGI)"):
    st.cache_data.clear()

try:
    fig_llegadas, fig_salidas, a_lleg, a_sali = fetch_and_plot_turismo()

    # --- SECCIÓN 1: LLEGADAS ---
    if fig_llegadas:
        st.subheader("✈️ 1. Llegadas de Turistas Internacionales")
        st.plotly_chart(fig_llegadas, use_container_width=True)
        
        # Tabla de datos visible
        st.markdown("**Datos detallados (Llegadas):**")
        table_lleg = prepare_table_data(a_lleg, PLOT_LAST_YEARS)
        st.dataframe(
            table_lleg,
            use_container_width=True,
            hide_index=True,
            column_config={
                "Año": st.column_config.NumberColumn(format="%d"),
                "Número de Turistas": st.column_config.NumberColumn(format="%,.0f")
            }
        )

    st.markdown("---") # Separador visual

    # --- SECCIÓN 2: SALIDAS ---
    if fig_salidas:
        st.subheader("🧳 2. Salidas de Residentes al Extranjero")
        st.plotly_chart(fig_salidas, use_container_width=True)
        
        # Tabla de datos visible
        st.markdown("**Datos detallados (Salidas):**")
        table_sali = prepare_table_data(a_sali, PLOT_LAST_YEARS)
        st.dataframe(
            table_sali,
            use_container_width=True,
            hide_index=True,
            column_config={
                "Año": st.column_config.NumberColumn(format="%d"),
                "Número de Turistas": st.column_config.NumberColumn(format="%,.0f")
            }
        )

except Exception as e:
    st.error(f"Error al procesar o obtener datos: {e}")